In [2]:
#https://liquipedia.net/starcraft2/Unit_Statistics_(Legacy_of_the_Void)
from bs4 import BeautifulSoup

In [461]:
def extract_property(ele):
  if(len(ele.text.strip()) == 0):
    try: 
      if(ele.find("a") is None):
#         print("No A element", ele)
        pass
      return ele.find("a").get("title")
    except:
      try: 
        if "Icon_Hitpoints" in ele.find("img").get("src"):
          return "Health"
        elif "Icon_Shields" in ele.find("img").get("src"):
          return "Shields"
        else:
          raise("row not recognized")
      except:
        return ""
  else:
    return ele.text.strip()

In [470]:
races = {"Protoss": protoss, "Terran":terran, "Zerg":zerg}
data = []
  
for title, race in races.items():
  header = None
  table_body = BeautifulSoup(race, 'html.parser')
  rows = table_body.find_all('tr')
  for row in rows:
      cols = row.find_all('td')
      if(len(cols) == 0):
        cols = row.find_all('th')
      cols = [extract_property(ele) for ele in cols]
      if header is None:
        header = cols
      if len(cols) > 0 and header != cols:
        unit_data = dict(zip(header, cols))
        unit_data["faction"] = title
        print(unit_data["Unit"], "|", unit_data["Health"])
        data.append(unit_data)

Probe | 20
Zealot | 100
Sentry | 40
Stalker | 80
Adept | 70
High Templar | 40
Dark Templar | 40
Archon | 10
Observer | 40
Warp Prism | 80
Immortal | 200
Colossus | 200
Disruptor | 100
Phoenix | 120
Void Ray | 150
Oracle | 100
Tempest | 200
Carrier | 300
Interceptor | 40
Mothership | 350
Photon Cannon | 150
SCV | 45
MULE | 60
Marine | 45 (+10)
Marauder | 125
Reaper | 60
Ghost | 100
Hellion | 90
Hellbat | 135
Widow Mine | 90
Siege Tank tm | 175
Siege Tank sm | 175
Cyclone | 120
Thor (Explosive) | 400
Thor (High Impact) | 400
Viking fm | 125
Viking am | 125
Medivac | 150
Liberator fm | 180
Liberator dm | 180
Banshee | 140
Raven | 140
Battlecruiser | 550
Planetary Fortress | 1500
Missile Turret | 250
Auto-Turret | 150
Larva | 25
Drone | 40
Queen | 175
Zergling | 35
Baneling | 30 (+5)
Roach | 145
Ravager | 120
Hydralisk | 90
Lurker | 200
Infestor | 90
Swarm Host | 160
Locust | 50
Ultralisk | 500
Overlord | 200
Overseer | 200
Changeling | 5
Mutalisk | 120
Corruptor | 200
Viper | 150
Brood Lo

In [463]:
import re , json
attributes_str = r"L - Light, A - Armored, B - Biological, M - Mechanical, P - Psionic, Ma - Massive, S - Structure, H - Heroic"
inner_dict = re.sub(r",?(\w+)\s-\s(\w+),?", "\"\\1\": \"\\2\",", attributes_str)[:-1]
attributes = json.loads("{{{}}}".format(inner_dict))

In [500]:
class Unit:
  def __init__(self, unit):
    self.name = unit["Unit"]
    self.faction = unit["faction"]
    self.types = Unit.get_types(unit)
    armor = Unit.extract_research(unit["Armor"])
    
    self.base = {}    
    self.base["armor"] = armor["base"]
    self.base["health"] = unit["Health"].split(" ")[0]
    if "Plasma Shield" in unit.keys():
      self.base["shields"] = int(unit["Plasma Shield"])
    for key in self.base.keys():
      self.base[key] = int(self.base[key])
    
    
    self.research = {}
    self.research["armor"] = int(armor["research"])
    
    self.attacks = []
    for a in ["G. Attack", "A. Attack"]:
      if unit[a] != "-":
        self.attacks.append(Unit.fetch_attack(unit, a))
    

  def get_types(unit):
    return [attributes[x] for x in unit["Attr."].split(", ")]
  
  def extract_research(attr):
    armor_str = re.sub("(\w*)\s\(\+(\w*)\)?(\sx(\w))?", "{\"base\":\\1, \"research\": \\2, \"repeats\":\"\\4\"}", attr)
    str_dict = json.loads(armor_str)
    return str_dict
    
  def fetch_attack(unit, attack_type):
    attack = {}
    attack["name"] = attack_type
    extraction = Unit.extract_research(unit[attack_type])
    attack["researchBonus"] = extraction["research"]
    attack["baseDamage"]= extraction["base"]
    if extraction["repeats"] == "":
      repeats = 1
    else:
      repeats = int(extraction["repeats"])
    attack["repeats"] = repeats
    
    attack["bonuses"] = []
    bonus = unit["Bonus"]
    if bonus != "-" and "," not in bonus and "/" not in bonus:
      b = json.loads(re.sub(
             "(\+([0-9]*))(\s\(\+([0-9]*)\))?(\s(\w*))?", 
             "{\"baseDamage\":\\2, \"researchBonus\": \"\\4\", \"to\":\"\\6\"}", 
             bonus))

      if b["researchBonus"] == "":
        b["researchBonus"] = 0
      else:
        b["researchBonus"] = int(b["researchBonus"])

      if "to" in b.keys():
        b["to"] = attributes[b["to"]]
      
      attack["bonuses"].append(b)

    return attack

In [501]:
def fetch_unit(name):
  return [x for x in data if x['Unit'].lower().startswith(name.lower())]

unit = fetch_unit("marine")[0]
u = Unit(unit)

In [502]:
u.__dict__

{'name': 'Marine',
 'faction': 'Terran',
 'types': ['Light', 'Biological'],
 'base': {'armor': 0, 'health': 45},
 'research': {'armor': 1},
 'attacks': [{'name': 'G. Attack',
   'researchBonus': 1,
   'baseDamage': 6,
   'repeats': 1,
   'bonuses': []},
  {'name': 'A. Attack',
   'researchBonus': 1,
   'baseDamage': 6,
   'repeats': 1,
   'bonuses': []}]}

In [503]:
marauder = Unit(fetch_unit("Maraud")[0])
immortal = Unit(fetch_unit("Immortal")[0])
normal_tank = Unit(fetch_unit("Siege")[0])
marine = Unit(fetch_unit("marine")[0])

assert(marauder.name == "Marauder")
assert(immortal.faction == "Protoss")
assert("Armored" in normal_tank.types)
assert(immortal.base["health"] == 200)
assert("shields" not in normal_tank.base.keys())

In [505]:
marauder.__dict__

{'name': 'Marauder',
 'faction': 'Terran',
 'types': ['Armored', 'Biological'],
 'base': {'armor': 1, 'health': 125},
 'research': {'armor': 1},
 'attacks': [{'name': 'G. Attack',
   'researchBonus': 1,
   'baseDamage': 5,
   'repeats': 2,
   'bonuses': [{'baseDamage': 5, 'researchBonus': 0, 'to': 'Armored'}]}]}

In [85]:
zerg = """<table style="text-align:center; white-space:nowrap;" class="wikitable sortable jquery-tablesorter">
<thead><tr>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Unit
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Supply" title="Supply"><img alt="" src="/commons/images/0/02/Supply-zerg.gif" width="16" height="16" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Minerals" title="Minerals"><img alt="" src="/commons/images/a/a4/Minerals.gif" width="14" height="14" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th width="3%" data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Gas" title="Gas"><img alt="" src="/commons/images/e/e4/Vespene-zerg.gif" width="16" height="16" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Game_Speed" title="Game Speed"><img alt="" src="/commons/images/7/7b/Buildtime_zerg.gif" width="18" height="16" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Size
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Cargo
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Armor" title="Armor"><img alt="" src="/commons/images/5/5e/Icon_Armor.png" width="16" height="15" loading="lazy"></a>
</th>
<th width="3%" data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><img alt="" src="/commons/images/e/e2/Icon_Hitpoints.png" width="16" height="16" loading="lazy">
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attr.
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. <abbr title="Damage">DPS</abbr>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. DPS
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus DPS
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attack Mod
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Cooldown" title="Cooldown">Cooldown</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Speed" title="Speed">Speed</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Creep" title="Creep">Creep</a> bonus
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Range" title="Range">Range</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Sight" title="Sight">Sight</a>
</th></tr></thead><tbody>
<tr>
<td><a href="/starcraft2/Larva_(Legacy_of_the_Void)" title="Larva (Legacy of the Void)">Larva</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>11
</td>
<td>0.25
</td>
<td>-
</td>
<td>10
</td>
<td>25
</td>
<td>L, B
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>(0.79)
</td>
<td>x1.0
</td>
<td>-
</td>
<td>5
</td></tr>
<tr>
<td><a href="/starcraft2/Drone_(Legacy_of_the_Void)" title="Drone (Legacy of the Void)">Drone</a>
</td>
<td>1
</td>
<td>50
</td>
<td>0
</td>
<td>12
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>40
</td>
<td>L, B
</td>
<td>5 (+0)
</td>
<td>-
</td>
<td>-
</td>
<td>4.7
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.07
</td>
<td>3.94
</td>
<td>x1.0
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Queen_(Legacy_of_the_Void)" title="Queen (Legacy of the Void)">Queen</a>
</td>
<td>2
</td>
<td>150
</td>
<td>0
</td>
<td>36
</td>
<td>1.75
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>175
</td>
<td>B, P
</td>
<td>4 (+1) x2
</td>
<td>9 (+1)
</td>
<td>-
</td>
<td>11.3 (+2.82)
</td>
<td>12.7 (+1.41)
</td>
<td>-
</td>
<td>P
</td>
<td>0.71
</td>
<td>1.31
</td>
<td>x2.7
</td>
<td>5, 7
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Zergling_(Legacy_of_the_Void)" title="Zergling (Legacy of the Void)">Zergling</a>
</td>
<td>0.5
</td>
<td>25
</td>
<td>0
</td>
<td>17
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>35
</td>
<td>L, B
</td>
<td>5 (+1)
</td>
<td>-
</td>
<td>-
</td>
<td>10 (+2), 14.1 (+2.82)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.497 (0.354)
</td>
<td>4.13 (+2.45)
</td>
<td>x1.3
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Baneling_(Legacy_of_the_Void)" title="Baneling (Legacy of the Void)">Baneling</a>
</td>
<td>0.5
</td>
<td>25
</td>
<td>25
</td>
<td>14
</td>
<td>0.75
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>30 (+5)
</td>
<td>B
</td>
<td>16 (+2)
</td>
<td>-
</td>
<td>+19 (+2) L / +64 (+5) S
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>S
</td>
<td>-
</td>
<td>3.5 (+0.63)
</td>
<td>x1.3
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Roach_(Legacy_of_the_Void)" title="Roach (Legacy of the Void)">Roach</a>
</td>
<td>2
</td>
<td>75
</td>
<td>25
</td>
<td>19
</td>
<td>1
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>145
</td>
<td>A, B
</td>
<td>16 (+2)
</td>
<td>-
</td>
<td>-
</td>
<td>11.2 (+1.4)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.43
</td>
<td>3.15 (+1.05)
</td>
<td>x1.3
</td>
<td>4
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Ravager" title="Ravager">Ravager</a>
</td>
<td>3
</td>
<td>25
</td>
<td>75
</td>
<td>9
</td>
<td>1.5
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>120
</td>
<td>B
</td>
<td>16 (+2)
</td>
<td>-
</td>
<td>-
</td>
<td>14 (+1.75)
</td>
<td>-
</td>
<td>-
</td>
<td>
</td>
<td>1.14
</td>
<td>3.85
</td>
<td>x1.3
</td>
<td>6
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Hydralisk_(Legacy_of_the_Void)" title="Hydralisk (Legacy of the Void)">Hydralisk</a>
</td>
<td>2
</td>
<td>100
</td>
<td>50
</td>
<td>24
</td>
<td>1.25
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>90
</td>
<td>L, B
</td>
<td>12 (+1)
</td>
<td>12 (+1)
</td>
<td>-
</td>
<td>22.2 (+1.85)
</td>
<td>22.2 (+1.85)
</td>
<td>-
</td>
<td>P
</td>
<td>0.54
</td>
<td>3.15 (+0.79)
</td>
<td>x1.3
</td>
<td>5 (+1)
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Lurker_(Legacy_of_the_Void)" title="Lurker (Legacy of the Void)">Lurker</a>
</td>
<td>3
</td>
<td>50
</td>
<td>100
</td>
<td>18
</td>
<td>1.5
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>200
</td>
<td>A, B
</td>
<td>20 (+2)
</td>
<td>-
</td>
<td>+10 (+1) A
</td>
<td>13.99 (+1.4)
</td>
<td>-
</td>
<td>+6.99 (+0.7) A
</td>
<td>L
</td>
<td>1.43
</td>
<td>4.13 (+0.413)
</td>
<td>x1.3
</td>
<td>8 (+2)
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Infestor_(Legacy_of_the_Void)" title="Infestor (Legacy of the Void)">Infestor</a>
</td>
<td>2
</td>
<td>100
</td>
<td>150
</td>
<td>36
</td>
<td>1.5
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>90
</td>
<td>A, B, P
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.15
</td>
<td>x1.3
</td>
<td>-
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Swarm_Host_(Legacy_of_the_Void)" title="Swarm Host (Legacy of the Void)">Swarm Host</a>
</td>
<td>3
</td>
<td>100
</td>
<td>75
</td>
<td>29
</td>
<td>1.625
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>160
</td>
<td>A, B
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>46.6 (+4.66)<sup id="cite_ref-5" class="reference" data-original-title="" title=""><a href="#">[z 1]</a></sup>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.15
</td>
<td>x1.3
</td>
<td>-
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Locust_(Legacy_of_the_Void)" title="Locust (Legacy of the Void)">Locust</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3
</td>
<td>0.75
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>50
</td>
<td>L, B
</td>
<td>10 (+1)
</td>
<td>-
</td>
<td>-
</td>
<td>23.3 (+2.33)
</td>
<td>-
</td>
<td>-
</td>
<td>P
</td>
<td>0.43
</td>
<td>2.62
</td>
<td>x1.4
</td>
<td>3
</td>
<td>6
</td></tr>
<tr>
<td><a href="/starcraft2/Ultralisk_(Legacy_of_the_Void)" title="Ultralisk (Legacy of the Void)">Ultralisk</a>
</td>
<td>6
</td>
<td>300
</td>
<td>200
</td>
<td>39
</td>
<td>2
</td>
<td>8
</td>
<td>2 (+1) ((+2))
</td>
<td>500
</td>
<td>A, B, Ma
</td>
<td>35 (+3)
</td>
<td>-
</td>
<td>-
</td>
<td>57.4 (+4.92)
</td>
<td>-
</td>
<td>-
</td>
<td>S
</td>
<td>0.61
</td>
<td>4.13 (+0.82)
</td>
<td>x1.3
</td>
<td>1
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Overlord_(Legacy_of_the_Void)" title="Overlord (Legacy of the Void)">Overlord</a>
</td>
<td>-
</td>
<td>100
</td>
<td>0
</td>
<td>18
</td>
<td>2
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>200
</td>
<td>A, B
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.902 (+1.728)
</td>
<td>-
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Overseer_(Legacy_of_the_Void)" title="Overseer (Legacy of the Void)">Overseer</a>
</td>
<td>-
</td>
<td>50
</td>
<td>50
</td>
<td>12
</td>
<td>2
</td>
<td>-
</td>
<td>1 (+1)
</td>
<td>200
</td>
<td>A, B
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>2.62 (+2.1)
</td>
<td>-
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Changeling_(Legacy_of_the_Void)" title="Changeling (Legacy of the Void)">Changeling</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1
</td>
<td>0.75
</td>
<td>-
</td>
<td>0
</td>
<td>5
</td>
<td>L, B
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.15
</td>
<td>x1.0
</td>
<td>-
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Mutalisk_(Legacy_of_the_Void)" title="Mutalisk (Legacy of the Void)">Mutalisk</a>
</td>
<td>2
</td>
<td>100
</td>
<td>100
</td>
<td>24
</td>
<td>1
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>120
</td>
<td>L, B
</td>
<td>9 (+1)
</td>
<td>9 (+1)
</td>
<td>-
</td>
<td>8.3 (+0.92)
</td>
<td>8.3 (+0.92)
</td>
<td>-
</td>
<td>B, P
</td>
<td>1.09
</td>
<td>5.6
</td>
<td>-
</td>
<td>3
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Corruptor_(Legacy_of_the_Void)" title="Corruptor (Legacy of the Void)">Corruptor</a>
</td>
<td>2
</td>
<td>150
</td>
<td>100
</td>
<td>29
</td>
<td>1.25
</td>
<td>-
</td>
<td>2 (+1)
</td>
<td>200
</td>
<td>A, B
</td>
<td>-
</td>
<td>14 (+1)
</td>
<td>+6 (+1) Ma
</td>
<td>-
</td>
<td>10.3 (+0.74)
</td>
<td>+4.4 (+0.74) Ma
</td>
<td>P
</td>
<td>1.36
</td>
<td>4.725
</td>
<td>-
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Viper_(Legacy_of_the_Void)" title="Viper (Legacy of the Void)">Viper</a>
</td>
<td>3
</td>
<td>100
</td>
<td>200
</td>
<td>29
</td>
<td>2
</td>
<td>-
</td>
<td>1 (+1)
</td>
<td>150
</td>
<td>A, B, P
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>4.13
</td>
<td>-
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Brood_Lord_(Legacy_of_the_Void)" title="Brood Lord (Legacy of the Void)">Brood Lord</a>
</td>
<td>4
</td>
<td>150
</td>
<td>150
</td>
<td>24
</td>
<td>2
</td>
<td>-
</td>
<td>1 (+1)
</td>
<td>225
</td>
<td>A, B, Ma
</td>
<td>20 (+2) (x2)
</td>
<td>-
</td>
<td>-
</td>
<td>22.4 (+2.24)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.79
</td>
<td>1.97
</td>
<td>-
</td>
<td>10
</td>
<td>12
</td></tr>
<tr>
<td><a href="/starcraft2/Broodling_(Legacy_of_the_Void)" title="Broodling (Legacy of the Void)">Broodling</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1
</td>
<td>0.75
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>30
</td>
<td>L, B
</td>
<td>4 (+1)
</td>
<td>-
</td>
<td>-
</td>
<td>8.7 (+2.17)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.46
</td>
<td>5.37
</td>
<td>x1.0
</td>
<td>M
</td>
<td>7
</td></tr>
<tr>
<td><a href="/starcraft2/Spine_Crawler_(Legacy_of_the_Void)" title="Spine Crawler (Legacy of the Void)">Spine Crawler</a>
</td>
<td>-
</td>
<td>100
</td>
<td>-
</td>
<td>36
</td>
<td>2x2
</td>
<td>-
</td>
<td>2
</td>
<td>300
</td>
<td>A, B, S
</td>
<td>25
</td>
<td>-
</td>
<td>+5 A
</td>
<td>18.9
</td>
<td>-
</td>
<td>+3.8 A
</td>
<td>-
</td>
<td>1.32
</td>
<td>0, 1.4
</td>
<td>x2.5
</td>
<td>7
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Spore_Crawler_(Legacy_of_the_Void)" title="Spore Crawler (Legacy of the Void)">Spore Crawler</a>
</td>
<td>-
</td>
<td>75
</td>
<td>-
</td>
<td>21
</td>
<td>2x2
</td>
<td>-
</td>
<td>1
</td>
<td>400
</td>
<td>A, B, S
</td>
<td>-
</td>
<td>15
</td>
<td>+15 B
</td>
<td>-
</td>
<td>24.6
</td>
<td>+24.6 B
</td>
<td>P
</td>
<td>0.61
</td>
<td>0, 1.4
</td>
<td>x2.6
</td>
<td>7
</td>
<td>11
</td></tr></tbody><tfoot></tfoot></table>"""

In [62]:
terran = """<table style="text-align:center; white-space:nowrap;" class="wikitable sortable jquery-tablesorter">
<thead><tr>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Unit
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Supply" title="Supply"><img alt="" src="/commons/images/b/ba/Supply-terran.gif" width="17" height="15" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Minerals" title="Minerals"><img alt="" src="/commons/images/a/a4/Minerals.gif" width="14" height="14" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th width="3%" data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Gas" title="Gas"><img alt="" src="/commons/images/8/8c/Vespene-terran.gif" width="14" height="18" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Game_Speed" title="Game Speed"><img alt="" src="/commons/images/0/0b/Buildtime_terran.gif" width="18" height="16" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Size
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Cargo
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Armor" title="Armor"><img alt="" src="/commons/images/5/5e/Icon_Armor.png" width="16" height="15" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><img alt="" src="/commons/images/e/e2/Icon_Hitpoints.png" width="16" height="16" loading="lazy">
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attr.
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. <abbr title="Damage">DPS</abbr>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. DPS
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus DPS
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attack Mod
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Cooldown" title="Cooldown">Cooldown</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Speed" title="Speed">Speed</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Range" title="Range">Range</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Sight" title="Sight">Sight</a>
</th></tr></thead><tbody>
<tr>
<td><a href="/starcraft2/SCV_(Legacy_of_the_Void)" title="SCV (Legacy of the Void)">SCV</a>
</td>
<td>1
</td>
<td>50
</td>
<td>0
</td>
<td>12
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>45
</td>
<td>L, B, M
</td>
<td>5 (+0)
</td>
<td>-
</td>
<td>-
</td>
<td>4.7
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.07
</td>
<td>3.94
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/MULE_(Legacy_of_the_Void)" title="MULE (Legacy of the Void)">MULE</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.75
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>60
</td>
<td>L, M
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.94
</td>
<td>-
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Marine_(Legacy_of_the_Void)" title="Marine (Legacy of the Void)">Marine</a>
</td>
<td>1
</td>
<td>50
</td>
<td>0
</td>
<td>18
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>45 (+10)
</td>
<td>L, B
</td>
<td>6 (+1)
</td>
<td>6 (+1)
</td>
<td>-
</td>
<td>9.8 (+1.64), 15 (+2.5)
</td>
<td>9.8 (+1.64), 15 (+2.5)
</td>
<td>-
</td>
<td>-
</td>
<td>0.61 (0.40)
</td>
<td>3.15 (+1.57)
</td>
<td>5
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Marauder_(Legacy_of_the_Void)" title="Marauder (Legacy of the Void)">Marauder</a>
</td>
<td>2
</td>
<td>100
</td>
<td>25
</td>
<td>21
</td>
<td>1.125
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>125
</td>
<td>A, B
</td>
<td>5 (+1) x2
</td>
<td>-
</td>
<td>+5 A
</td>
<td>9.3 (+1.87), 14.1 (+2.82)
</td>
<td>-
</td>
<td>+9.3, +14.1 A
</td>
<td>P
</td>
<td>1.07 (0.71)
</td>
<td>3.15 (+1.57)
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Reaper_(Legacy_of_the_Void)" title="Reaper (Legacy of the Void)">Reaper</a>
</td>
<td>1
</td>
<td>50
</td>
<td>50
</td>
<td>32
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>60
</td>
<td>L, B
</td>
<td>4 (+1) x2
</td>
<td>-
</td>
<td>-
</td>
<td>10.1 (+2.53)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.79
</td>
<td>5.25
</td>
<td>5
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Ghost_(Legacy_of_the_Void)" title="Ghost (Legacy of the Void)">Ghost</a>
</td>
<td>2
</td>
<td>150
</td>
<td>125
</td>
<td>29
</td>
<td>0.75
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>100
</td>
<td>B, P
</td>
<td>10 (+1)
</td>
<td>10 (+1)
</td>
<td>+10 (+1) L
</td>
<td>9.3 (+0.93)
</td>
<td>9.3 (+0.93)
</td>
<td>+9.3 (+0.93) L
</td>
<td>-
</td>
<td>1.07
</td>
<td>3.94
</td>
<td>6
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Hellion_(Legacy_of_the_Void)" title="Hellion (Legacy of the Void)">Hellion</a>
</td>
<td>2
</td>
<td>100
</td>
<td>0
</td>
<td>21
</td>
<td>1.25
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>90
</td>
<td>L, M
</td>
<td>8 (+1)
</td>
<td>-
</td>
<td>+6 (+1), +5 L
</td>
<td>4.5 (+0.56)
</td>
<td>-
</td>
<td>+3.4 (+0.56), 6.16 (+0.56) L
</td>
<td>L
</td>
<td>1.79
</td>
<td>5.95
</td>
<td>5
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Hellbat_(Legacy_of_the_Void)" title="Hellbat (Legacy of the Void)">Hellbat</a>
</td>
<td>2
</td>
<td>100
</td>
<td>0
</td>
<td>21
</td>
<td>1.25
</td>
<td>4
</td>
<td>0 (+1)
</td>
<td>135
</td>
<td>L, B, M
</td>
<td>18 (+2)
</td>
<td>-
</td>
<td>+0 (+1), +12 L
</td>
<td>12.6 (+1.4)
</td>
<td>-
</td>
<td>+0 (+0.7), 8.4 (+0.7) L
</td>
<td>S
</td>
<td>1.43
</td>
<td>3.15
</td>
<td>2
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Widow_Mine_(Legacy_of_the_Void)" title="Widow Mine (Legacy of the Void)">Widow Mine</a>
</td>
<td>2
</td>
<td>75
</td>
<td>25
</td>
<td>21
</td>
<td>1
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>90
</td>
<td>L, M
</td>
<td>125
</td>
<td>-
</td>
<td>+35 Shield
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>S
</td>
<td>29
</td>
<td>3.94
</td>
<td>5
</td>
<td>7
</td></tr>
<tr>
<td><a href="/starcraft2/Siege_Tank_(Legacy_of_the_Void)" title="Siege Tank (Legacy of the Void)">Siege Tank</a> tm
</td>
<td>3
</td>
<td>150
</td>
<td>125
</td>
<td>32
</td>
<td>1.75
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>175
</td>
<td>A, M
</td>
<td>15 (+2)
</td>
<td>-
</td>
<td>+10 (+1) A
</td>
<td>20.3 (+2.7)
</td>
<td>-
</td>
<td>+13.5 (+1.35) A
</td>
<td>-
</td>
<td>0.74
</td>
<td>3.15
</td>
<td>7
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Siege_Tank_(Legacy_of_the_Void)" title="Siege Tank (Legacy of the Void)">Siege Tank</a> sm
</td>
<td>3
</td>
<td>150
</td>
<td>125
</td>
<td>32
</td>
<td>1.75
</td>
<td>8
</td>
<td>1 (+1)
</td>
<td>175
</td>
<td>A, M
</td>
<td>40 (+4)
</td>
<td>-
</td>
<td>30 (+1) A
</td>
<td>18.69 (+2.6)
</td>
<td>-
</td>
<td>+14.02 (+0.87) A
</td>
<td>S
</td>
<td>2.14
</td>
<td>-
</td>
<td>13
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Cyclone" title="Cyclone">Cyclone</a>
</td>
<td>3
</td>
<td>150
</td>
<td>100
</td>
<td>32
</td>
<td>1.5
</td>
<td>3
</td>
<td>1 (+1)
</td>
<td>120
</td>
<td>A, M
</td>
<td>18 (+2)
</td>
<td>18 (+2)
</td>
<td>-
</td>
<td>25.2 (+2.8)
</td>
<td>25.2 (+2.8)
</td>
<td>-
</td>
<td>P
</td>
<td>0.71
</td>
<td>4.72
</td>
<td>5
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Thor_(Legacy_of_the_Void)" title="Thor (Legacy of the Void)">Thor</a> (Explosive)
</td>
<td>6
</td>
<td>300
</td>
<td>200
</td>
<td>43
</td>
<td>2
</td>
<td>8
</td>
<td>1 (+1)
</td>
<td>400
</td>
<td>A, M, Ma
</td>
<td>30 (+3) x2
</td>
<td>6 (+1) (x4)
</td>
<td>+6 (+1) L Air
</td>
<td>65.9 (+6.59)
</td>
<td>11.2 (+1.87)
</td>
<td>+11.2 (+1.87) L Air
</td>
<td>S, P (Air)
</td>
<td>0.91, 2.14
</td>
<td>2.62
</td>
<td>7, 10 Air
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Thor_(Legacy_of_the_Void)" title="Thor (Legacy of the Void)">Thor</a> (High Impact)
</td>
<td>6
</td>
<td>300
</td>
<td>200
</td>
<td>43
</td>
<td>2
</td>
<td>8
</td>
<td>1 (+1)
</td>
<td>400
</td>
<td>A, M, Ma
</td>
<td>30 (+3) x2
</td>
<td>25 (+4)
</td>
<td>+10 (+1) Ma Air
</td>
<td>65.9 (+6.59)
</td>
<td>27.5 (+3.3)
</td>
<td>+11.0 (+1.1) Ma Air
</td>
<td>P (Air)
</td>
<td>0.91
</td>
<td>2.62
</td>
<td>7, 11 Air
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Viking_(Legacy_of_the_Void)" title="Viking (Legacy of the Void)">Viking</a> fm
</td>
<td>2
</td>
<td>150
</td>
<td>75
</td>
<td>30
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>125
</td>
<td>A, M
</td>
<td>-
</td>
<td>10 (+1) x2
</td>
<td>+4 A
</td>
<td>-
</td>
<td>14 (+1.4)
</td>
<td>+5.59 A
</td>
<td>P
</td>
<td>1.43
</td>
<td>3.85
</td>
<td>9
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Viking_(Legacy_of_the_Void)" title="Viking (Legacy of the Void)">Viking</a> am
</td>
<td>2
</td>
<td>150
</td>
<td>75
</td>
<td>30
</td>
<td>1.5
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>125
</td>
<td>A, M
</td>
<td>12 (+1)
</td>
<td>-
</td>
<td>+8 (+1) M
</td>
<td>16.9 (+1.41)
</td>
<td>-
</td>
<td>+11.3 (+1.41) M
</td>
<td>-
</td>
<td>0.71
</td>
<td>3.15
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Medivac_(Legacy_of_the_Void)" title="Medivac (Legacy of the Void)">Medivac</a>
</td>
<td>2
</td>
<td>100
</td>
<td>100
</td>
<td>30
</td>
<td>1.5
</td>
<td>-
</td>
<td>1 (+1)
</td>
<td>150
</td>
<td>A, M
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.5 (0.63) (+2.44)
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Liberator" title="Liberator">Liberator</a> fm
</td>
<td>3
</td>
<td>150
</td>
<td>150
</td>
<td>43
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>180
</td>
<td>A, M
</td>
<td>-
</td>
<td>5 (+1) x2
</td>
<td>-
</td>
<td>-
</td>
<td>7.75 (+1.55)
</td>
<td>-
</td>
<td>P
</td>
<td>1.29
</td>
<td>4.72
</td>
<td>5
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Liberator" title="Liberator">Liberator</a> dm
</td>
<td>3
</td>
<td>150
</td>
<td>150
</td>
<td>43
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>180
</td>
<td>A, M
</td>
<td>75 (+5)
</td>
<td>-
</td>
<td>-
</td>
<td>65.8 (+4.39)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.14
</td>
<td>-
</td>
<td>10 (+3)
</td>
<td>10 (+3)
</td></tr>
<tr>
<td><a href="/starcraft2/Banshee_(Legacy_of_the_Void)" title="Banshee (Legacy of the Void)">Banshee</a>
</td>
<td>3
</td>
<td>150
</td>
<td>100
</td>
<td>43
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>140
</td>
<td>L, M
</td>
<td>12 (+1) x2
</td>
<td>-
</td>
<td>-
</td>
<td>27 (+2.25)
</td>
<td>-
</td>
<td>-
</td>
<td>P
</td>
<td>0.89
</td>
<td>3.85 (+1.4)
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Raven_(Legacy_of_the_Void)" title="Raven (Legacy of the Void)">Raven</a>
</td>
<td>2
</td>
<td>100
</td>
<td>200
</td>
<td>43
</td>
<td>1.25
</td>
<td>-
</td>
<td>1 (+1)
</td>
<td>140
</td>
<td>L, M
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>4.13
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Battlecruiser_(Legacy_of_the_Void)" title="Battlecruiser (Legacy of the Void)">Battlecruiser</a>
</td>
<td>6
</td>
<td>400
</td>
<td>300
</td>
<td>64
</td>
<td>2.5
</td>
<td>-
</td>
<td>3 (+1)
</td>
<td>550
</td>
<td>A, M, Ma
</td>
<td>8 (+1)
</td>
<td>6 (+1)
</td>
<td>-
</td>
<td>50 (+6.25)
</td>
<td>37.5 (+6.25)
</td>
<td>-
</td>
<td>-
</td>
<td>0.16
</td>
<td>2.62
</td>
<td>6
</td>
<td>12
</td></tr>
<tr>
<td><a href="/starcraft2/Planetary_Fortress_(Legacy_of_the_Void)" title="Planetary Fortress (Legacy of the Void)">Planetary Fortress</a>
</td>
<td>-
</td>
<td>150
</td>
<td>150
</td>
<td>36
</td>
<td>5x5
</td>
<td>-
</td>
<td>3 (+2)
</td>
<td>1500
</td>
<td>A, M, S
</td>
<td>40
</td>
<td>-
</td>
<td>-
</td>
<td>28
</td>
<td>-
</td>
<td>-
</td>
<td>S
</td>
<td>1.43
</td>
<td>-
</td>
<td>6 (+1)
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Missile_Turret_(Legacy_of_the_Void)" title="Missile Turret (Legacy of the Void)">Missile Turret</a>
</td>
<td>-
</td>
<td>100
</td>
<td>0
</td>
<td>18
</td>
<td>2x2
</td>
<td>-
</td>
<td>0 (+2)
</td>
<td>250
</td>
<td>A, M, S
</td>
<td>-
</td>
<td>12 x2
</td>
<td>-
</td>
<td>-
</td>
<td>39.3
</td>
<td>-
</td>
<td>-
</td>
<td>0.61
</td>
<td>-
</td>
<td>7 (+1)
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Auto-Turret_(Legacy_of_the_Void)" title="Auto-Turret (Legacy of the Void)">Auto-Turret</a>
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>2x2
</td>
<td>-
</td>
<td>1 (+2)
</td>
<td>150
</td>
<td>A, M, S
</td>
<td>18
</td>
<td>18
</td>
<td>-
</td>
<td>31.58
</td>
<td>31.58
</td>
<td>-
</td>
<td>-
</td>
<td>0.57
</td>
<td>-
</td>
<td>6 (+1)
</td>
<td>7
</td></tr></tbody><tfoot></tfoot></table>"""

In [1]:
protoss = """<table style="text-align:center; white-space:nowrap;" class="wikitable sortable jquery-tablesorter">
<thead><tr>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Unit
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Supply" title="Supply"><img alt="" src="/commons/images/8/86/Supply-protoss.gif" width="16" height="16" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Minerals" title="Minerals"><img alt="" src="/commons/images/a/a4/Minerals.gif" width="14" height="14" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Gas" title="Gas"><img alt="" src="/commons/images/2/2e/Vespene-protoss.gif" width="15" height="15" style="vertical-align: baseline" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Game_Speed" title="Game Speed"><img alt="" src="/commons/images/a/a4/Buildtime_protoss.gif" width="18" height="16" style="vertical-align: baseline" loading="lazy"></a> / <a href="/starcraft2/Warp_Gate" class="mw-redirect" title="Warp Gate">WG</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Size
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Cargo
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Armor" title="Armor"><img alt="" src="/commons/images/5/5e/Icon_Armor.png" width="16" height="15" loading="lazy"></a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><img alt="" src="/commons/images/e/e2/Icon_Hitpoints.png" width="16" height="16" loading="lazy">
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Plasma_Shield" title="Plasma Shield"><img alt="" src="/commons/images/8/82/Icon_Shields.png" width="16" height="16" loading="lazy"></a>
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attr.
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. Attack
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">G. <abbr title="Damage"><a href="/starcraft2/DPS" class="mw-redirect" title="DPS">DPS</a></abbr>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">A. DPS
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Bonus DPS
</th>
<th class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending">Attack Mod
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Cooldown" title="Cooldown">Cooldown</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Speed" title="Speed">Speed</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Range" title="Range">Range</a>
</th>
<th data-sort-type="number" class="headerSort" tabindex="0" role="columnheader button" title="Sort ascending"><a href="/starcraft2/Sight" title="Sight">Sight</a>
</th></tr></thead><tbody>
<tr>
<td><a href="/starcraft2/Probe_(Legacy_of_the_Void)" title="Probe (Legacy of the Void)">Probe</a>
</td>
<td>1
</td>
<td>50
</td>
<td>0
</td>
<td>12
</td>
<td>0.75
</td>
<td>1
</td>
<td>0 (+1)
</td>
<td>20
</td>
<td>20
</td>
<td>L, M
</td>
<td>5 (+0)
</td>
<td>-
</td>
<td>-
</td>
<td>4.7
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.07
</td>
<td>3.94
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Zealot_(Legacy_of_the_Void)" title="Zealot (Legacy of the Void)">Zealot</a>
</td>
<td>2
</td>
<td>100
</td>
<td>0
</td>
<td>27/20
</td>
<td>1
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>100
</td>
<td>50
</td>
<td>L, B
</td>
<td>8 (+1) x2
</td>
<td>-
</td>
<td>-
</td>
<td>18.6 (+2.33)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>0.86
</td>
<td>3.15 (4.725 +5.67 C)
</td>
<td>M
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Sentry_(Legacy_of_the_Void)" title="Sentry (Legacy of the Void)">Sentry</a>
</td>
<td>2
</td>
<td>50
</td>
<td>100
</td>
<td>26/23
</td>
<td>1
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>40
</td>
<td>40
</td>
<td>L, M, P
</td>
<td>6 (+1)
</td>
<td>6 (+1)
</td>
<td>-
</td>
<td>8.5 (+1.41)
</td>
<td>8.5 (+1.41)
</td>
<td>-
</td>
<td>-
</td>
<td>0.71
</td>
<td>3.15
</td>
<td>5
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Stalker_(Legacy_of_the_Void)" title="Stalker (Legacy of the Void)">Stalker</a>
</td>
<td>2
</td>
<td>125
</td>
<td>50
</td>
<td>30/23
</td>
<td>1.25
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>80
</td>
<td>80
</td>
<td>A, M
</td>
<td>13 (+1)
</td>
<td>13 (+1)
</td>
<td>+5 (+1) A
</td>
<td>9.7 (+0.75)
</td>
<td>9.7 (+0.75)
</td>
<td>+3.7 (+0.75) A
</td>
<td>P
</td>
<td>1.34
</td>
<td>4.13
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Adept" title="Adept">Adept</a>
</td>
<td>2
</td>
<td>100
</td>
<td>25
</td>
<td>30/20
</td>
<td>1
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>70
</td>
<td>70
</td>
<td>L, B
</td>
<td>10 (+1)
</td>
<td>-
</td>
<td>+12 (+1) L
</td>
<td>6.2 (+0.62)
</td>
<td>-
</td>
<td>+7.45 (+0.62) L
</td>
<td>-
</td>
<td>1.61 (1.11)
</td>
<td>3.5 (4.78)
</td>
<td>4
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/High_Templar_(Legacy_of_the_Void)" title="High Templar (Legacy of the Void)">High Templar</a>
</td>
<td>2
</td>
<td>50
</td>
<td>150
</td>
<td>39/32
</td>
<td>0.75
</td>
<td>2
</td>
<td>0 (+1)
</td>
<td>40
</td>
<td>40
</td>
<td>L, B, P
</td>
<td>4 (+1)
</td>
<td>-
</td>
<td>-
</td>
<td>3.2 (+0.8)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.25
</td>
<td>2.62
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Dark_Templar_(Legacy_of_the_Void)" title="Dark Templar (Legacy of the Void)">Dark Templar</a>
</td>
<td>2
</td>
<td>125
</td>
<td>125
</td>
<td>39/32
</td>
<td>0.75
</td>
<td>2
</td>
<td>1 (+1)
</td>
<td>40
</td>
<td>80
</td>
<td>L, B, P
</td>
<td>45 (+5)
</td>
<td>-
</td>
<td>-
</td>
<td>37.2 (+4.13)
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>1.21
</td>
<td>3.94
</td>
<td>M
</td>
<td>8
</td></tr>
<tr>
<td><a href="/starcraft2/Archon_(Legacy_of_the_Void)" title="Archon (Legacy of the Void)">Archon</a>
</td>
<td>4
</td>
<td>0
</td>
<td>0
</td>
<td>8.57
</td>
<td>2
</td>
<td>4
</td>
<td>0 (+1)
</td>
<td>10
</td>
<td>350
</td>
<td>P, Ma
</td>
<td>25 (+3)
</td>
<td>25 (+3)
</td>
<td>+10 (+1) B
</td>
<td>20 (+2.4)
</td>
<td>20 (+2.4)
</td>
<td>+8 (+0.8) B
</td>
<td>S
</td>
<td>1.25
</td>
<td>3.94
</td>
<td>3
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Observer_(Legacy_of_the_Void)" title="Observer (Legacy of the Void)">Observer</a>
</td>
<td>1
</td>
<td>25
</td>
<td>75
</td>
<td>21
</td>
<td>1
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>40
</td>
<td>20
</td>
<td>L, M
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>2.63 (+1.31)
</td>
<td>-
</td>
<td>11
</td></tr>
<tr>
<td><a href="/starcraft2/Warp_Prism_(Legacy_of_the_Void)" title="Warp Prism (Legacy of the Void)">Warp Prism</a>
</td>
<td>2
</td>
<td>250
</td>
<td>0
</td>
<td>36
</td>
<td>1.75
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>80
</td>
<td>100
</td>
<td>A, M, P
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>4.13 (5.36)
</td>
<td>-
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Immortal_(Legacy_of_the_Void)" title="Immortal (Legacy of the Void)">Immortal</a>
</td>
<td>4
</td>
<td>275
</td>
<td>100
</td>
<td>39
</td>
<td>1.5
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>200
</td>
<td>100
</td>
<td>A, M
</td>
<td>20 (+2)
</td>
<td>-
</td>
<td>+30 (+3) A
</td>
<td>19.2 (+1.92)
</td>
<td>-
</td>
<td>+28.8 (+2.88) A
</td>
<td>-
</td>
<td>1.04
</td>
<td>3.15
</td>
<td>6
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Colossus_(Legacy_of_the_Void)" title="Colossus (Legacy of the Void)">Colossus</a>
</td>
<td>6
</td>
<td>300
</td>
<td>200
</td>
<td>54
</td>
<td>2
</td>
<td>8
</td>
<td>1 (+1)
</td>
<td>200
</td>
<td>150
</td>
<td>A, M, Ma
</td>
<td>10 (+1) x2
</td>
<td>-
</td>
<td>+5 (+1) L
</td>
<td>18.7 (+1.87)
</td>
<td>-
</td>
<td>+9.3 (+1.87) L
</td>
<td>L
</td>
<td>1.07
</td>
<td>3.15
</td>
<td>7 (+2)
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Disruptor" title="Disruptor">Disruptor</a>
</td>
<td>3
</td>
<td>150
</td>
<td>150
</td>
<td>36
</td>
<td>1
</td>
<td>4
</td>
<td>1 (+1)
</td>
<td>100
</td>
<td>100
</td>
<td>A, M
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>3.15
</td>
<td>-
</td>
<td>9
</td></tr>
<tr>
<td><a href="/starcraft2/Phoenix_(Legacy_of_the_Void)" title="Phoenix (Legacy of the Void)">Phoenix</a>
</td>
<td>2
</td>
<td>150
</td>
<td>100
</td>
<td>25
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>120
</td>
<td>60
</td>
<td>L, M
</td>
<td>-
</td>
<td>5 (+1) x2
</td>
<td>+5 L
</td>
<td>-
</td>
<td>12.6 (+2.52)
</td>
<td>+12.6 L
</td>
<td>P
</td>
<td>0.79
</td>
<td>5.95
</td>
<td>5 (+2)
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Void_Ray_(Legacy_of_the_Void)" title="Void Ray (Legacy of the Void)">Void Ray</a>
</td>
<td>4
</td>
<td>200
</td>
<td>150
</td>
<td>37
</td>
<td>2
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>150
</td>
<td>100
</td>
<td>A, M
</td>
<td>6 (+1)
</td>
<td>6 (+1)
</td>
<td>+4, +10 A
</td>
<td>16.8 (+2.8)
</td>
<td>16.8 (+2.8)
</td>
<td>+11.2, +28 A
</td>
<td>-
</td>
<td>0.36
</td>
<td>3.85 (+0.798), (2.888 (-0.264)) <sup id="cite_ref-prismaticalignment_1-0" class="reference" data-original-title="" title=""><a href="#">[p 1]</a></sup>
</td>
<td>6
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Oracle_(Legacy_of_the_Void)" title="Oracle (Legacy of the Void)">Oracle</a>
</td>
<td>3
</td>
<td>150
</td>
<td>150
</td>
<td>37
</td>
<td>1.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>100
</td>
<td>60
</td>
<td>A, M, P
</td>
<td>15<sup id="cite_ref-2" class="reference" data-original-title="" title=""><a href="#">[p 2]</a></sup>
</td>
<td>-
</td>
<td>+7 L
</td>
<td>24.4
</td>
<td>-
</td>
<td>+11.5 L
</td>
<td>-
</td>
<td>0.61
</td>
<td>5.6
</td>
<td>4
</td>
<td>10
</td></tr>
<tr>
<td><a href="/starcraft2/Tempest_(Legacy_of_the_Void)" title="Tempest (Legacy of the Void)">Tempest</a>
</td>
<td>5
</td>
<td>250
</td>
<td>175
</td>
<td>43
</td>
<td>2.5
</td>
<td>-
</td>
<td>2 (+1)
</td>
<td>200
</td>
<td>100
</td>
<td>A, M, Ma
</td>
<td>40 (+4)
</td>
<td>30 (+3)
</td>
<td>+22 (+2) Ma Air, +0 (+40) S
</td>
<td>16.97 (+1.697)
</td>
<td>12.7 (+1.27)
</td>
<td>+9.32 (+0.847) Ma Air, +0 (+16.97) S
</td>
<td>P
</td>
<td>2.36
</td>
<td>3.15
</td>
<td>10, 14
</td>
<td>12
</td></tr>
<tr>
<td><a href="/starcraft2/Carrier_(Legacy_of_the_Void)" title="Carrier (Legacy of the Void)">Carrier</a>
</td>
<td>6
</td>
<td>350
</td>
<td>250
</td>
<td>64
</td>
<td>2.5
</td>
<td>-
</td>
<td>2 (+1)
</td>
<td>300
</td>
<td>150
</td>
<td>A, M, Ma
</td>
<td>-
</td>
<td>-
</td>
<td>-
</td>
<td>37.3 (+7.5)<sup id="cite_ref-interceptors_3-0" class="reference" data-original-title="" title=""><a href="#">[p 3]</a></sup>
</td>
<td>37.3 (+7.5)
</td>
<td>-
</td>
<td>-
</td>
<td>0.36 (0.09/ 0.18)<sup id="cite_ref-gravitoncatapult_4-0" class="reference" data-original-title="" title=""><a href="#">[p 4]</a></sup>
</td>
<td>2.62
</td>
<td>8
</td>
<td>12
</td></tr>
<tr>
<td><a href="/starcraft2/Interceptor_(Legacy_of_the_Void)" title="Interceptor (Legacy of the Void)">Interceptor</a>
</td>
<td>0
</td>
<td>15
</td>
<td>0
</td>
<td>11
</td>
<td>0.5
</td>
<td>-
</td>
<td>0 (+1)
</td>
<td>40
</td>
<td>40
</td>
<td>L, M
</td>
<td>5 (+1) x2
</td>
<td>5 (+1) x2
</td>
<td>-
</td>
<td>4.7 (+0.93)
</td>
<td>4.7 (+0.93)
</td>
<td>-
</td>
<td>-
</td>
<td>2.14
</td>
<td>10.5
</td>
<td>2
</td>
<td>7
</td></tr>
<tr>
<td><a href="/starcraft2/Mothership_(Legacy_of_the_Void)" title="Mothership (Legacy of the Void)">Mothership</a>
</td>
<td>8
</td>
<td>400
</td>
<td>400
</td>
<td>114
</td>
<td>2.75
</td>
<td>-
</td>
<td>2 (+1)
</td>
<td>350
</td>
<td>350
</td>
<td>A, M, P, Ma, H
</td>
<td>6 (+1) x6
</td>
<td>6 (+1) x6
</td>
<td>-
</td>
<td>22.8 (+3.78)
</td>
<td>22.8 (+3.78)
</td>
<td>-
</td>
<td>-
</td>
<td>1.58
</td>
<td>2.62
</td>
<td>7
</td>
<td>14
</td></tr>
<tr>
<td><a href="/starcraft2/Photon_Cannon_(Legacy_of_the_Void)" title="Photon Cannon (Legacy of the Void)">Photon Cannon</a>
</td>
<td>-
</td>
<td>150
</td>
<td>0
</td>
<td>29
</td>
<td>2x2
</td>
<td>-
</td>
<td>1
</td>
<td>150
</td>
<td>150
</td>
<td>A, S
</td>
<td>20
</td>
<td>20
</td>
<td>-
</td>
<td>22.5
</td>
<td>22.5
</td>
<td>-
</td>
<td>-
</td>
<td>0.89
</td>
<td>-
</td>
<td>7
</td>
<td>11
</td></tr></tbody><tfoot></tfoot></table>"""